# Техники градиентного бустинга, бэггинга и стэкинга для решения задач классификации и регрессии
Группа: ИВТ-М20.<br/>
Студент: Лискунов Роман Геннадьвич

In [1]:
from math import isnan
from pprint import pprint
from typing import (
    Any,
    Union
)
from warnings import filterwarnings

from pandas import (
    DataFrame,
    read_csv,
    Series,
    read_table
)
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    StackingClassifier
)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    accuracy_score
)
from sklearn.model_selection import (
    train_test_split, GridSearchCV
)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

In [2]:
filterwarnings('ignore')

## Прочтите данные из файлов

In [3]:
sop1: DataFrame = DataFrame(
    read_table(
        "data/train/SOP1.dat",
        sep="\s",
        header=0
    )
)
sop1 = sop1.drop('0000', axis=1)
sop1.name = "SOP1"
sop1.shape

(1042, 16)

In [4]:
sop2: DataFrame = DataFrame(
    read_table(
        "data/train/SOP2.dat",
        sep="\s",
        header=0
    )
)
sop2 = sop2.drop('0000', axis=1)
sop2.name = "SOP2"
sop2.shape

(1039, 16)

In [5]:
sop3: DataFrame = DataFrame(
    read_table(
        "data/train/SOP3.dat",
        sep="\s",
        header=0
    )
)
sop3 = sop3.drop('0000', axis=1)
sop3.name = "SOP3"
sop3.shape

(1038, 16)

In [6]:
target1: DataFrame = DataFrame(
    read_csv(
        "data/train/target1.csv",
        sep=",",
        header=0
    )
)
target1.name = "target1"

In [7]:
target2: DataFrame = DataFrame(
    read_csv(
        "data/train/target2.csv",
        sep=",",
        header=0
    )
)
target2.name = "target2"

In [8]:
target3: DataFrame = DataFrame(
    read_csv(
        "data/train/target3.csv",
        sep=",",
        header=0
    )
)
target3.name = "target3"

##  Отобразите несколько первых и несколько последних записей.

In [9]:
def printer(variable: DataFrame) -> None:
    pprint(variable.name)
    print(end="\n")
    pprint(variable.head())
    print(end="\n\n")
    pprint(variable.tail())
    print(end="\n\n\n\n")

In [10]:
printer(sop1)

'SOP1'

   29-08-03  12:34:21  +20C  Unnamed: 4  Unnamed: 5  257  Unnamed: 7  \
0         0        35   202          27          29   35           0   
1         0        23   190          27          22   29           0   
2         1         0   188          24          10   13           0   
3         5         0   183          26           6    1           0   
4         4         0   188          32           4    0           0   

   Unnamed: 8  325  Unnamed: 10  Unnamed: 11   16  Unnamed: 13  Unnamed: 14  \
0           0  255          147           54  119          125          212   
1           0  255          147           56   95          156          223   
2           0  255          145           59   40          172          254   
3           0  255          153           66   22          172          255   
4           0  255          157           62   18          170          255   

    5  СОП1  
0  17    39  
1  13    27  
2  10     0  
3   7     0  
4  14    21  


In [11]:
printer(sop2)

'SOP2'

   29-08-03  13:00:27  +23C  Unnamed: 4  Unnamed: 5  257  Unnamed: 7  \
0         0         0   175          12           0    0           0   
1         0         0   159          10          10    0           0   
2         0         0   139          25          48    0           0   
3         0         0   130          54          61    0           0   
4         0         0   110          54          57    0           0   

   Unnamed: 8  325  Unnamed: 10  Unnamed: 11  16  Unnamed: 13  Unnamed: 14  \
0           0  255          112            0   0          233          173   
1           0  255          108            0   0          222          173   
2           0  255          145            0   0          230          175   
3           0  255          216            0   1          237          185   
4           0  236          207            3   4          239          205   

    5  СОП2  
0   2    23  
1  10    34  
2  18    18  
3  52    22  
4  43    20  


    

In [12]:
printer(sop3)

'SOP3'

   29-08-03  13:49:13  +21C  Unnamed: 4  Unnamed: 5  257  Unnamed: 7  \
0         0         0     0          10         148   56           0   
1         0         0     0          28         129   55           0   
2         0         0     0          48          94   58          13   
3         0         0     0          54          68   51           0   
4         0         0     0          81          46   36           0   

   Unnamed: 8  325  Unnamed: 10  Unnamed: 11  16  Unnamed: 13  Unnamed: 14  \
0           3   39          118            0   4           88          154   
1           0   48          150            0   0           67          140   
2           0   16          210            0   0           47           98   
3           0   18          244            2   0           37           73   
4           0   16          255           11   0           41           58   

    5  СОП3  
0  49    14  
1  54     5  
2  59    62  
3  29    12  
4  15     3  


    

In [13]:
printer(target1)

'target1'

   Начало дефекта  Длина дефекта  Высота дефекта Тип дефекта Сторона
0              51           42.0               7           L       A
1             200           40.0               6           L       B
2             278           39.0               4           L       A
3             365           43.0               6           L       B
4             460           27.0               5           L       A


   Начало дефекта  Длина дефекта  Высота дефекта Тип дефекта Сторона
3             365           43.0               6           L       B
4             460           27.0               5           L       A
5             845           36.0               4           L       B
6             959           62.0               6           L       A
7             141            NaN               4           T        






In [14]:
printer(target2)

'target2'

   Начало дефекта  Длина дефекта  Высота дефекта Тип дефекта Сторона
0             178           37.0               6           L       B
1             258           38.0               4           L       A
2             337           53.0               5           L       B
3             822           37.0               5           L       B
4             957           48.0               5           L       A


   Начало дефекта  Длина дефекта  Высота дефекта Тип дефекта Сторона
1             258           38.0               4           L       A
2             337           53.0               5           L       B
3             822           37.0               5           L       B
4             957           48.0               5           L       A
5             123            NaN               3           T        






In [15]:
printer(target3)

'target3'

   Начало дефекта  Длина дефекта  Высота дефекта Тип дефекта Сторона
0              26           42.0               5           L       A
1             172           47.0               6           L       B
2             276           55.0               8           L       B
3             795           34.0               8           L       B
4             877           49.0               4           L       A


   Начало дефекта  Длина дефекта  Высота дефекта Тип дефекта Сторона
2             276           55.0               8           L       B
3             795           34.0               8           L       B
4             877           49.0               4           L       A
5             971           50.0               5           L       B
6             112            NaN               4           T        






## Вектор с результатами наличия дефектов

In [16]:
array1: Union[Union[list[int], None], Any] = [0] * sop1.count()[0]
array2: Union[Union[list[int], None], Any] = [0] * sop2.count()[0]
array3: Union[Union[list[int], None], Any] = [0] * sop3.count()[0]

In [17]:
ser1: Series = Series(data=array1)
ser2: Series = Series(data=array2)
ser3: Series = Series(data=array3)

In [18]:
def defects(target: DataFrame, series: Series) -> Series:
    for index, row in target.iterrows():
        begin = row['Начало дефекта']
        end = row['Начало дефекта'] + row['Длина дефекта']
        if not isnan(end):
            series.loc[begin:int(end)] = 1
    return series

In [19]:
ser1 = defects(target1, ser1)
ser2 = defects(target2, ser2)
ser3 = defects(target2, ser3)

## Разбиение данных на обучающую и проверочную выборки

In [20]:
sop_train1, sop_test1, target_train1, target_test1 = train_test_split(sop1, ser1, test_size=0.3, train_size=0.7)
sop_train2, sop_test2, target_train2, target_test2 = train_test_split(sop2, ser2, test_size=0.3, train_size=0.7)
sop_train3, sop_test3, target_train3, target_test3 = train_test_split(sop3, ser3, test_size=0.3, train_size=0.7)

## Последовательное применение алгоритмов

In [21]:
def algorithms(sop_train: object, target_train: object) -> object:
    rfc: RandomForestClassifier = RandomForestClassifier()
    rfc.fit(
        X=sop_train,
        y=target_train
    )

    gbc: GradientBoostingClassifier = GradientBoostingClassifier()
    gbc.fit(
        X=sop_train,
        y=target_train
    )

    estimators = [
        (
            'rf', RandomForestClassifier(
                n_estimators=10,
                random_state=42
            )
        ),
        (
            'svr', make_pipeline(
                StandardScaler(),
                LinearSVC(random_state=42)
            )
        )
    ]
    sc: StackingClassifier = StackingClassifier(
        estimators=estimators,
        final_estimator=LogisticRegression()
    )
    sc.fit(sop_train, target_train)

    return rfc, gbc, sc

In [22]:
rfc1, gbc1, sc1 = algorithms(
    sop_train=sop_train1,
    target_train=target_train1
)

In [23]:
rfc2, gbc2, sc2 = algorithms(
    sop_train=sop_train2,
    target_train=target_train2
)

In [24]:
rfc3, gbc3, sc3 = algorithms(
    sop_train=sop_train3,
    target_train=target_train3
)

## Оценка качества модели

In [25]:
def estimate(classifier: object, target_test: object, sop_test: object) -> None:
    pred: object = classifier.predict(sop_test)
    print(f"accuracy_score: {round(accuracy_score(target_test, pred, normalize=True), 3)}", end='\n\n')
    print(f"classification_report: {classification_report(target_test, pred)}", end='\n\n')

### RandomForestClassifier

In [26]:
estimate(
    classifier=rfc1,
    target_test=target_test1,
    sop_test=sop_test1
)

accuracy_score: 0.869

classification_report:               precision    recall  f1-score   support

           0       0.89      0.94      0.91       227
           1       0.81      0.69      0.74        86

    accuracy                           0.87       313
   macro avg       0.85      0.81      0.83       313
weighted avg       0.87      0.87      0.87       313




In [27]:
estimate(
    classifier=rfc2,
    target_test=target_test2,
    sop_test=sop_test2
)

accuracy_score: 0.885

classification_report:               precision    recall  f1-score   support

           0       0.88      0.98      0.93       248
           1       0.89      0.50      0.64        64

    accuracy                           0.88       312
   macro avg       0.89      0.74      0.79       312
weighted avg       0.89      0.88      0.87       312




In [28]:
estimate(
    classifier=rfc3,
    target_test=target_test3,
    sop_test=sop_test3
)

accuracy_score: 0.894

classification_report:               precision    recall  f1-score   support

           0       0.88      1.00      0.94       243
           1       0.97      0.54      0.69        69

    accuracy                           0.89       312
   macro avg       0.93      0.77      0.81       312
weighted avg       0.90      0.89      0.88       312




### GradientBoostingClassifier

In [29]:
estimate(
    classifier=gbc1,
    target_test=target_test1,
    sop_test=sop_test1
)

accuracy_score: 0.84

classification_report:               precision    recall  f1-score   support

           0       0.87      0.91      0.89       227
           1       0.74      0.65      0.69        86

    accuracy                           0.84       313
   macro avg       0.81      0.78      0.79       313
weighted avg       0.84      0.84      0.84       313




In [30]:
estimate(
    classifier=gbc2,
    target_test=target_test2,
    sop_test=sop_test2
)

accuracy_score: 0.872

classification_report:               precision    recall  f1-score   support

           0       0.88      0.98      0.92       248
           1       0.83      0.47      0.60        64

    accuracy                           0.87       312
   macro avg       0.86      0.72      0.76       312
weighted avg       0.87      0.87      0.86       312




In [31]:
estimate(
    classifier=gbc3,
    target_test=target_test3,
    sop_test=sop_test3
)

accuracy_score: 0.894

classification_report:               precision    recall  f1-score   support

           0       0.88      1.00      0.94       243
           1       0.97      0.54      0.69        69

    accuracy                           0.89       312
   macro avg       0.93      0.77      0.81       312
weighted avg       0.90      0.89      0.88       312




### StackingClassifier

In [32]:
estimate(
    classifier=sc1,
    target_test=target_test1,
    sop_test=sop_test1
)

accuracy_score: 0.875

classification_report:               precision    recall  f1-score   support

           0       0.90      0.93      0.92       227
           1       0.80      0.73      0.76        86

    accuracy                           0.88       313
   macro avg       0.85      0.83      0.84       313
weighted avg       0.87      0.88      0.87       313




In [33]:
estimate(
    classifier=sc2,
    target_test=target_test2,
    sop_test=sop_test2
)

accuracy_score: 0.878

classification_report:               precision    recall  f1-score   support

           0       0.90      0.96      0.93       248
           1       0.77      0.58      0.66        64

    accuracy                           0.88       312
   macro avg       0.83      0.77      0.79       312
weighted avg       0.87      0.88      0.87       312




In [34]:
estimate(
    classifier=sc3,
    target_test=target_test3,
    sop_test=sop_test3
)

accuracy_score: 0.891

classification_report:               precision    recall  f1-score   support

           0       0.88      1.00      0.93       243
           1       0.97      0.52      0.68        69

    accuracy                           0.89       312
   macro avg       0.93      0.76      0.81       312
weighted avg       0.90      0.89      0.88       312




## Подбор гиперпараметров

In [35]:
def hyperparameters(sop_train: object, target_train: object) -> object:
    print("RandomForestClassifier")
    parameters: dict[str, Union[list[str], list[Union[float, int]]]] = {
        'criterion': ['gini', 'entropy'],
        'max_depth': [10, 12],
        'min_samples_split': [1, 2, 3, 4],
    }
    grid: GridSearchCV = GridSearchCV(
        RandomForestClassifier(),
        parameters,
        cv=5
    )
    rfc_grid: Union[GridSearchCV, Any] = grid.fit(sop_train, target_train)

    print("GradientBoostingClassifier")
    parameters: dict[str, Union[list[str], list[Union[float, int]]]] = {
        "n_estimators": [100, 150, 200],
        "learning_rate": [0.1, 0.3, 0.5],
        "max_depth": [1, 2],
    }
    grid: GridSearchCV = GridSearchCV(
        GradientBoostingClassifier(),
        parameters,
        cv=5
    )
    gbc_grid: Union[GridSearchCV, Any] = grid.fit(sop_train, target_train)

    print("StackingClassifier")
    estimators: list[Union[tuple[str, RandomForestClassifier], tuple[str, LogisticRegression]]] = [
        (
            'rf', RandomForestClassifier(
                n_estimators=10,
                random_state=42
            )
        ),
        (
            'logreg',
            LogisticRegression()
        )
    ]
    params: dict[str, list[int]] = {
        'rf__n_estimators': [5, 10, 15]
    }
    grid: GridSearchCV = GridSearchCV(
        estimator=StackingClassifier(
            estimators=estimators,
            final_estimator=DecisionTreeClassifier()
        ),
        param_grid=params,
        cv=5
    )
    sc_grid: Union[GridSearchCV, Any] = grid.fit(sop_train, target_train)

    return rfc_grid, gbc_grid, sc_grid

In [36]:
rfc_grid1, gbc_grid1, sc_grid1 = hyperparameters(
    sop_train=sop_train1,
    target_train=target_train1
)

RandomForestClassifier
GradientBoostingClassifier
StackingClassifier


In [37]:
rfc_grid2, gbc_grid2, sc_grid2 = hyperparameters(
    sop_train=sop_train2,
    target_train=target_train2
)

RandomForestClassifier
GradientBoostingClassifier
StackingClassifier


In [38]:
rfc_grid3, gbc_grid3, sc_grid3 = hyperparameters(
    sop_train=sop_train3,
    target_train=target_train3
)

RandomForestClassifier
GradientBoostingClassifier
StackingClassifier


## Заново обучите модель с подобранными гиперпараметрам

### RandomForestClassifier

In [39]:
estimate(
    classifier=rfc_grid1,
    target_test=target_test1,
    sop_test=sop_test1
)

accuracy_score: 0.888

classification_report:               precision    recall  f1-score   support

           0       0.89      0.96      0.93       227
           1       0.87      0.70      0.77        86

    accuracy                           0.89       313
   macro avg       0.88      0.83      0.85       313
weighted avg       0.89      0.89      0.88       313




In [40]:
estimate(
    classifier=rfc_grid2,
    target_test=target_test2,
    sop_test=sop_test2
)

accuracy_score: 0.888

classification_report:               precision    recall  f1-score   support

           0       0.89      0.98      0.93       248
           1       0.89      0.52      0.65        64

    accuracy                           0.89       312
   macro avg       0.89      0.75      0.79       312
weighted avg       0.89      0.89      0.88       312




In [41]:
estimate(
    classifier=rfc_grid3,
    target_test=target_test3,
    sop_test=sop_test3
)

accuracy_score: 0.891

classification_report:               precision    recall  f1-score   support

           0       0.88      1.00      0.93       243
           1       1.00      0.51      0.67        69

    accuracy                           0.89       312
   macro avg       0.94      0.75      0.80       312
weighted avg       0.90      0.89      0.88       312




### GradientBoostingClassifier

In [42]:
estimate(
    classifier=gbc_grid1,
    target_test=target_test1,
    sop_test=sop_test1
)

accuracy_score: 0.84

classification_report:               precision    recall  f1-score   support

           0       0.88      0.90      0.89       227
           1       0.72      0.67      0.70        86

    accuracy                           0.84       313
   macro avg       0.80      0.79      0.80       313
weighted avg       0.84      0.84      0.84       313




In [43]:
estimate(
    classifier=gbc_grid2,
    target_test=target_test2,
    sop_test=sop_test2
)

accuracy_score: 0.878

classification_report:               precision    recall  f1-score   support

           0       0.90      0.95      0.93       248
           1       0.75      0.61      0.67        64

    accuracy                           0.88       312
   macro avg       0.83      0.78      0.80       312
weighted avg       0.87      0.88      0.87       312




In [44]:
estimate(
    classifier=gbc_grid3,
    target_test=target_test3,
    sop_test=sop_test3
)

accuracy_score: 0.894

classification_report:               precision    recall  f1-score   support

           0       0.90      0.98      0.93       243
           1       0.88      0.61      0.72        69

    accuracy                           0.89       312
   macro avg       0.89      0.79      0.83       312
weighted avg       0.89      0.89      0.89       312




### StackingClassifier

In [45]:
estimate(
    classifier=sc_grid1,
    target_test=target_test1,
    sop_test=sop_test1
)

accuracy_score: 0.837

classification_report:               precision    recall  f1-score   support

           0       0.90      0.87      0.89       227
           1       0.68      0.76      0.72        86

    accuracy                           0.84       313
   macro avg       0.79      0.81      0.80       313
weighted avg       0.84      0.84      0.84       313




In [46]:
estimate(
    classifier=sc_grid2,
    target_test=target_test2,
    sop_test=sop_test2
)

accuracy_score: 0.795

classification_report:               precision    recall  f1-score   support

           0       0.88      0.86      0.87       248
           1       0.50      0.53      0.52        64

    accuracy                           0.79       312
   macro avg       0.69      0.70      0.69       312
weighted avg       0.80      0.79      0.80       312




In [47]:
estimate(
    classifier=sc_grid3,
    target_test=target_test3,
    sop_test=sop_test3
)

accuracy_score: 0.853

classification_report:               precision    recall  f1-score   support

           0       0.89      0.93      0.91       243
           1       0.69      0.59      0.64        69

    accuracy                           0.85       312
   macro avg       0.79      0.76      0.77       312
weighted avg       0.85      0.85      0.85       312




# Ответы на контрольные вопросы

## 1. Что такое алгоритм «Случайного леса»? Чем он отличается от обычного дерева решений?

Случайные леса — это метод обучения ансамбля, который работает путем построения множества деревьев решений во время обучения и вывода класса в зависимости от отдельных деревьев.

Главное отличие между деревом решений и случайным лесом является то, что Дерево решений — это график, который использует метод ветвления, чтобы проиллюстрировать каждый возможный результат решения, в то время как случайный лес - это набор деревьев решений, который дает окончательный результат на основе выходных данных всех его деревьев решений.

## 2. Что такое алгоритм градиентного бустинга?

Бустинг — это техника построения ансамблей, в которой предсказатели построены не независимо, а последовательно.

Это техника использует идею о том, что следующая модель будет учится на ошибках предыдущей. Они имеют неравную вероятность появления в последующих моделях, и чаще появятся те, что дают наибольшую ошибку. Предсказатели могут быть выбраны из широкого ассортимента моделей, например, деревья решений, регрессия, классификаторы и т.д. Из-за того, что предсказатели обучаются на ошибках, совершенных предыдущими, требуется меньше времени для того, чтобы добраться до реального ответа. Но мы должны выбирать критерий остановки с осторожностью, иначе это может привести к переобучению. Градиентный бустинг — это пример бустинга.

Градиентный бустинг — это техника машинного обучения для задач классификации и регрессии, которая строит модель предсказания в форме ансамбля слабых предсказывающих моделей, обычно деревьев решений.

Цель любого алгоритма обучения с учителем — определить функцию потерь и минимизировать её. Пусть, например, в качестве функции потерь будет среднеквадратичная ошибка (MSE).

Мы хотим, чтобы построить наши предсказания таким образом, чтобы MSE была минимальна. Используя градиентный спуск и обновляя предсказания, основанные на скорости обучения (learning rate), ищем значения, на которых MSE минимальна.

Итак, мы просто обновляем предсказания таким образом, что сумма наших отклонений стремилась к нулю и предсказанные значения были близки к реальным.

## 3. Что означает техника стекинга в машинном обучении?

Стековое обобщение (stacked generalization), или просто стекинг (stacking) — еще один способ объединения классификаторов, вводящий понятие мета-алгоритма обучения. В отличие от бэггинга и бустинга, при стекинге используются классификаторы разной природы. Идея стекинга такова:

1. разбить обучающую выборку на два непересекающихся подмножества;

2. обучить несколько базовых классификаторов на первом подмножестве;

3. тестировать базовые классификаторы на втором подмножестве;

4. используя предсказания из предыдущего пункта как входные данные, а истинные классы объектов как выход, обучить мета-алгоритм.